In [5]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()       # load environment variables from .env file

True

In [6]:
client = OpenAI(
    api_key=os.getenv("RUNPOD_TOKEN"),
    base_url=os.getenv("RUNPOD_CHATBOT_URL"),   #URL for the llama model deployed on runpod
)
model_name = os.getenv("MODEL_NAME")

In [ ]:
# Send a request to the model to generate a response
response = client.chat.completions.create(
    model=model_name,
    # The conversation messages to send to the model
    messages=[
        {"role": "user", "content": "What's the capital of Germany?"},  # The first message in a conversation, from the user
    ],
    # Temperature controls the creativity of the response
    # Lower values give more predictable answers, higher values give more creative answers
    temperature=0.0,  # 0.0 = factual, no randomness

    # top_p controls the diversity of responses. It looks at the most probable tokens
    # Lower values make the response more focused, higher values allow more randomness
    top_p=0.8,  # 0.8 = consider 80% of the most probable tokens for the response

    # max_tokens limits how long the model’s response can be
    # 2000 tokens is a large number, allowing long responses
    max_tokens=2000,  # Maximum length of the model's response (in tokens)
)

In [13]:
response

ChatCompletion(id='chatcmpl-9f429b1438d84f62879eeab303b2f5e1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of Germany is Berlin.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1741843570, model='meta-llama/Meta-Llama-3-8B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=8, prompt_tokens=17, total_tokens=25, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)

In [14]:
response.choices[0].message.content

'The capital of Germany is Berlin.'